In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import re

In [4]:
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [3]:
# remove unwanted characters in Tweets using Regex
def preProcess_data(text):
   text = text.lower()
   new_text = re.sub('[^a-zA-z0-9\s]','',text)
   new_text = re.sub('rt', '', new_text)
   return new_text


data['text'] = data['text'].apply(preProcess_data)

In [ ]:
#using Tensorflow’s tokenizer to tokenize our dataset, and Tensorflow’s pad_sequences to pad our sequences
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X, 28)

Y = pd.get_dummies(data['sentiment']).values

In [ ]:
#splitting the dataset into training and testing portions.
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20)

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(128,recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])


In [ ]:

batch_size = 512

model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, validation_data=(X_test, Y_test))

Epoch 1/10
22/22 [==============================] - 33s 1s/step - loss: 0.9448 - accuracy: 0.6065 - val_loss: 0.8902 - val_accuracy: 0.6079
Epoch 2/10
22/22 [==============================] - 27s 1s/step - loss: 0.8312 - accuracy: 0.6271 - val_loss: 0.7896 - val_accuracy: 0.6519
Epoch 3/10
22/22 [==============================] - 28s 1s/step - loss: 0.7418 - accuracy: 0.6730 - val_loss: 0.7557 - val_accuracy: 0.6714
Epoch 4/10
22/22 [==============================] - 27s 1s/step - loss: 0.6814 - accuracy: 0.7039 - val_loss: 0.7478 - val_accuracy: 0.6854
Epoch 5/10
22/22 [==============================] - 28s 1s/step - loss: 0.6413 - accuracy: 0.7236 - val_loss: 0.7382 - val_accuracy: 0.6825
Epoch 6/10
22/22 [==============================] - 27s 1s/step - loss: 0.6144 - accuracy: 0.7386 - val_loss: 0.7503 - val_accuracy: 0.6905
Epoch 7/10
22/22 [==============================] - 27s 1s/step - loss: 0.5879 - accuracy: 0.7444 - val_loss: 0.7730 - val_accuracy: 0.6825
Epoch 8/10
22/22 [==

In [ ]:
model.save('sentiment.h5')